In [10]:
%%html
<style>
.output_subarea.output_text.output_stream.output_stdout > pre {
   white-space: pre;
}
.p-Widget.jp-RenderedText.jp-OutputArea-output > pre {
   white-space: pre;
}
</style>

In [42]:
import pandas as pd
import numpy as np

import glob
import os
import json

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark import SparkContext
from pyspark.sql import functions as F


In [45]:
# sc = SparkContext('local', 'first')
# spark = SparkSession(sc)

In [44]:
spark = SparkSession.builder.master("local[1]") \
                    .appName('SparkByExamples.com') \
                    .getOrCreate()

# spark.stop()

In [46]:
bname = 't_mbmi_credit_balance'
# Read the schema
with open(f"schema/{bname}.json") as f:
    new_schema = StructType.fromJson(json.load(f))
    print(new_schema.simpleString())

struct<gl_branch_id:string,manager_id:string,customer_id:string,contract_id:string,account_id:string,sender_application_id:string,commercial_product_type:string,commercial_subproduct_type:string,origin_currency_id:string,account_opening_date:date,current_contract_expiration_date:date,contract_duration_number:int,cutoff_date:date,avg_bal_orig_curncy_amount:decimal(37,2),total_average_balance_amount:decimal(37,2),origin_currency_amount:decimal(37,2),overdue_bal_local_curncy_amount:decimal(37,2),current_bal_local_curncy_amount:decimal(37,2),overdue_balance_amount:decimal(37,2),current_balance_amount:decimal(37,2),gl_date:date>


In [47]:
df = spark.read.load("csv/t_mbmi_credit_balance.csv",
                     format="csv", sep="|", header="true", schema=new_schema)
df.count()

In [49]:
df.show()

+------------+----------+-----------+------------------+----------+---------------------+-----------------------+--------------------------+------------------+--------------------+--------------------------------+------------------------+-----------+--------------------------+----------------------------+----------------------+-------------------------------+-------------------------------+----------------------+----------------------+----------+
|gl_branch_id|manager_id|customer_id|       contract_id|account_id|sender_application_id|commercial_product_type|commercial_subproduct_type|origin_currency_id|account_opening_date|current_contract_expiration_date|contract_duration_number|cutoff_date|avg_bal_orig_curncy_amount|total_average_balance_amount|origin_currency_amount|overdue_bal_local_curncy_amount|current_bal_local_curncy_amount|overdue_balance_amount|current_balance_amount|   gl_date|
+------------+----------+-----------+------------------+----------+---------------------+---------

In [50]:
df.write.partitionBy("cutoff_date").format("parquet").save(f"{bname}", mode='overwrite')

Py4JJavaError: An error occurred while calling o198.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:110)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103)


In [52]:
df.write.format('parquet').partitionBy('cutoff_date').save(bname)

Py4JJavaError: An error occurred while calling o217.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:110)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103)


In [41]:
df.write.partitionBy("cutoff_date",).mode("overwrite").parquet(f"output/{bname}.parquet")

Py4JJavaError: An error occurred while calling o150.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:110)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103